In [1]:
# autoreload import your package
%load_ext autoreload
%autoreload 2

from make_dataset import *


In [2]:
cfg = ExtractConfig(max_examples=(10, 10), max_length=999)
cfg


ExtractConfig(datasets=('amazon_polarity', 'super_glue:boolq', 'glue:qnli', 'imdb'), model='TheBloke/WizardCoder-Python-13B-V1.0-GPTQ', data_dirs=(), max_examples=(10, 10), num_shots=1, num_variants=-1, layers=(), seed=42, token_loc='last', template_path=None, max_length=999)

In [3]:
model, tokenizer = load_model(cfg.model)


2023-10-15 16:46:07.158 | INFO     | src.models.load:verbose_change_param:18 - changing pad_token_id from 32000 to 0
2023-10-15 16:46:07.159 | INFO     | src.models.load:verbose_change_param:18 - changing padding_side from right to left
2023-10-15 16:46:07.159 | INFO     | src.models.load:verbose_change_param:18 - changing truncation_side from right to left


In [4]:
ds_name = cfg.datasets[0]
split_type = "train"
ds_tokens = load_preproc_dataset(ds_name, cfg, tokenizer)

ds_tokens_calibration = ds_tokens.select(range(10))


tokenize:   0%|          | 0/32 [00:00<?, ? examples/s]

truncated:   0%|          | 0/32 [00:00<?, ? examples/s]

prompt_truncated:   0%|          | 0/32 [00:00<?, ? examples/s]

choice_ids:   0%|          | 0/32 [00:00<?, ? examples/s]

Filter:   0%|          | 0/32 [00:00<?, ? examples/s]

removed truncated rows to leave: num_rows 10


In [5]:
# b = next(iter(ds_tokens))
# b


In [6]:
# get the calibration dataset
BATCH_SIZE = 2
f = None
info_kwargs = dict(extract_cfg=cfg.to_dict(), ds_name=ds_name, split_type=split_type, f=f, date=pd.Timestamp.now().isoformat(),)
intervention_dicts = [None, ]
gen_kwargs = dict(
    model=model,
    tokenizer=tokenizer,
    data=ds_tokens,
    batch_size=BATCH_SIZE,
    layer_padding=cfg.layer_padding,
    layer_stride=cfg.layer_stride,
    intervention_dicts=intervention_dicts,
)


In [7]:
ds1 = Dataset.from_generator(
    generator=batch_hidden_states,
    info=DatasetInfo(
        description=json.dumps(info_kwargs, indent=2),
        config_name=f,
    ),
    gen_kwargs=gen_kwargs,
    num_proc=1,
)
ds1


Generating train split: 0 examples [00:00, ? examples/s]

get hidden states:   0%|          | 0/5 [00:00<?, ?it/s]

Dataset({
    features: ['scores0', 'ds_index', 'scores', 'head_activation', 'ds_string', 'example_i', 'answer', 'question', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'truncated', 'prompt_truncated'],
    num_rows: 10
})

# Scratch


In [8]:
from src.datasets.hs import ExtractHiddenStates
ehs = ExtractHiddenStates(model, tokenizer, intervention_dicts=intervention_dicts, layer_stride=cfg.layer_stride, layer_padding=cfg.layer_padding)
ehs


ExtractHiddenStates(model=LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (rotary_emb): LlamaRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): LlamaMLP(
          (act_fn): SiLUActivation()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=5120, out_features=32001, bias=False)
), tokenizer=LlamaTokenizerFast(name_or_path='TheBloke/WizardCoder-Python-13B-V1.0-GPTQ', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left',

In [9]:
from torch.utils.data import DataLoader
batch_size = BATCH_SIZE
data = ds_tokens_calibration
from src.helpers.ds import ds_keep_cols, clear_mem

# get a batch
torch_cols = ['input_ids', 'attention_mask', 'choice_ids']
ds_t_subset = ds_keep_cols(data, torch_cols)
ds_t_subset.set_format(type='torch')

ds_p_subset = data.remove_columns(torch_cols)
dl = DataLoader(ds_t_subset, batch_size=batch_size, shuffle=False)
for i, batch in enumerate(tqdm(dl, desc='get hidden states')):
    input_ids, attention_mask, choice_ids =  batch["input_ids"], batch["attention_mask"], batch["choice_ids"]
    
#     nn = len(input_ids)
#     index = i*batch_size+np.arange(nn)
    
#     # different due to dropout
#     hsl = ehs.get_batch_of_hidden_states(input_ids=input_ids, attention_mask=attention_mask, choice_ids=choice_ids)


get hidden states:   0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
# from src.datasets.intervene import get_com_directions, get_interventions_dict
# head_wise_activations = np.array(ds1['head_activation'])
# labels=np.array(ds1["label_true"])
# get_com_directions(2, 
#                    2, 
#                    head_wise_activations, 
#                    labels
#                    )


In [11]:
from einops import rearrange, reduce, repeat, asnumpy, parse_shape
from src.datasets.intervene import InterventionDict
from typing import Tuple

activations = np.array(ds1['head_activation']).squeeze(-1)
labels = np.array(ds1["label_true"]).astype(int)==1
num_heads = model.config.num_attention_heads

layer_names = [f"model.layers.{i}.self_attn" for i in range(model.config.num_hidden_layers)]
layer_names, layer_inds = ehs.get_layer_selection(layer_names)


In [14]:
from functools import partial
from baukit.nethook import Trace, TraceDict, recursive_copy
from src.datasets.intervene import intervention_meta_fn, get_interventions_dict

interventions = get_interventions_dict(activations, labels, layer_names, num_heads)
intervention_fn = partial(intervention_meta_fn, interventions=interventions, num_heads=num_heads)
model.cuda().eval()

device = model.device
with torch.no_grad():
    with TraceDict(model, layer_names, edit_output=intervention_fn) as ret:
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), return_dict=True, output_hidden_states=True)
        a = outputs[0]
a


tensor([[[-3.4062, 12.5859,  0.4016,  ..., -2.1797, -1.2871, -2.1543],
         [-3.4082, 12.5625,  0.3992,  ..., -2.1777, -1.2900, -2.1543],
         [-3.4062, 12.5703,  0.3999,  ..., -2.1777, -1.2910, -2.1543],
         ...,
         [-2.4629,  0.2039, 12.3828,  ..., -2.2266,  0.8071,  0.2996],
         [-6.0508, -6.7305, 10.3047,  ..., -4.7031, -2.3340, -2.0586],
         [-7.2188, -8.3750,  9.3047,  ..., -5.4141, -2.8828, -3.3223]],

        [[-3.4004, 12.8203,  0.4661,  ..., -2.1914, -1.1826, -2.1445],
         [-3.4023, 12.7891,  0.4626,  ..., -2.1914, -1.1885, -2.1465],
         [-3.4004, 12.7969,  0.4636,  ..., -2.1914, -1.1865, -2.1445],
         ...,
         [-1.5488,  0.2537, 10.4453,  ..., -1.8154,  0.9443,  0.6328],
         [-6.2344, -7.9570,  7.3828,  ..., -4.1797, -3.3203, -2.4883],
         [-5.7266, -7.7461, 10.3828,  ..., -3.4277, -2.1797, -2.3887]]],
       device='cuda:0')

In [15]:
with torch.no_grad():
    with TraceDict(model, layer_names) as ret:
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), return_dict=True, output_hidden_states=True)
        a = outputs[0]
a


tensor([[[-3.4043, 12.5703,  0.4009,  ..., -2.1777, -1.2881, -2.1523],
         [-3.4043, 12.5781,  0.4006,  ..., -2.1758, -1.2881, -2.1523],
         [-3.4043, 12.5703,  0.4006,  ..., -2.1777, -1.2900, -2.1543],
         ...,
         [-2.4629,  0.2039, 12.3828,  ..., -2.2266,  0.8071,  0.2996],
         [-6.0508, -6.7305, 10.3047,  ..., -4.7031, -2.3340, -2.0586],
         [-6.3906, -7.0547,  8.7266,  ..., -4.8984, -2.3691, -2.8887]],

        [[-3.4023, 12.8203,  0.4658,  ..., -2.1934, -1.1855, -2.1445],
         [-3.4023, 12.7891,  0.4644,  ..., -2.1914, -1.1875, -2.1445],
         [-3.4023, 12.8125,  0.4648,  ..., -2.1934, -1.1865, -2.1445],
         ...,
         [-1.5488,  0.2537, 10.4453,  ..., -1.8154,  0.9443,  0.6328],
         [-6.2344, -7.9570,  7.3828,  ..., -4.1797, -3.3203, -2.4883],
         [-4.9375, -6.7891,  9.5312,  ..., -3.1250, -1.7793, -1.9893]]],
       device='cuda:0')

In [22]:
intervention_fn2 = partial(intervention_meta_fn, interventions=interventions, num_heads=num_heads, alpha=-15)
model.cuda().eval()

device = model.device
with torch.no_grad():
    with TraceDict(model, layer_names, edit_output=intervention_fn2) as ret:
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), return_dict=True, output_hidden_states=True)
        a = outputs[0]
a


tensor([[[-3.4062, 12.5859,  0.4014,  ..., -2.1797, -1.2881, -2.1543],
         [-3.4082, 12.5859,  0.4011,  ..., -2.1797, -1.2881, -2.1562],
         [-3.4062, 12.5703,  0.3999,  ..., -2.1777, -1.2910, -2.1543],
         ...,
         [-2.4629,  0.2039, 12.3828,  ..., -2.2266,  0.8071,  0.2996],
         [-6.0508, -6.7305, 10.3047,  ..., -4.7031, -2.3340, -2.0586],
         [-5.8555, -6.3125,  7.6172,  ..., -4.5625, -2.0273, -2.6055]],

        [[-3.4023, 12.8125,  0.4661,  ..., -2.1953, -1.1846, -2.1445],
         [-3.4023, 12.7891,  0.4639,  ..., -2.1914, -1.1885, -2.1445],
         [-3.4004, 12.7969,  0.4629,  ..., -2.1895, -1.1885, -2.1445],
         ...,
         [-1.5488,  0.2537, 10.4453,  ..., -1.8154,  0.9443,  0.6328],
         [-6.2344, -7.9570,  7.3828,  ..., -4.1797, -3.3203, -2.4883],
         [-4.5547, -6.3555,  7.9883,  ..., -3.1680, -1.6172, -1.8477]]],
       device='cuda:0')